## Parsing the Mozilla PSL
The PSL is composed of three sections:
- ICANN DOMAINS country code
- ICANN DOMAINS generic
- PRIVATE DOMAINS

The TLDs of the first two sections eTLDs should be included in the `tld-list` of IANA, which separates Country-Code and Generic TLD.

The last one instead is available only in the Mozilla list, because these eTLDs are less *standard*.

In [4]:
# getting the IANA list with category Country-Code and Generic

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os, datetime, time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import re
import json

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

DIR = os.path.dirname('.')

datasetFrame = pd.read_csv('/home/princio/Desktop/malware_detection/nn/nn/dataset_training.csv')
datasetFrame['tld'] = datasetFrame.dn.apply(lambda dn: dn[1 + dn.rfind('.'):])

def get_file(fname, fetch_func):
    if os.path.exists(fname):
        ts = os.path.getmtime(fname)
        tdelta = datetime.datetime.utcnow() - datetime.datetime.utcfromtimestamp(ts)
        if tdelta.days < 7:
            return
    flines = fetch_func(fname)
    with open(fname, 'w') as f:
        f.writelines(flines)
    return


get_file(
    'iana.csv',
    lambda fname: pd.read_html('https://www.iana.org/domains/root/db', attrs = {'id': 'tld-table'})[0].to_csv()
)
get_file(
    'tldlist.csv',
    lambda fname: pd.read_csv('https://tld-list.com/df/tld-list-details.csv').to_csv()
)
get_file(
    'public_suffix_list.dat',
    lambda fname: requests.get(
        'https://publicsuffix.org/list/public_suffix_list.dat',
        verify=False,
        allow_redirects=True,
        timeout=5
    ).text
)


df_iana = pd.read_csv('iana.csv', index_col=0)

In [7]:
# getting the IANA list with category Country-Code and Generic

df_iana = pd.read_csv('iana.csv', index_col=0)
df_tldlist = pd.read_csv('tldlist.csv', index_col=0)
        
df_iana = df_iana.rename(columns={
    'Domain': 'tld', 'Type': 'type', 'TLD Manager': 'manager'
})

df_tldlist = df_tldlist.rename(columns={'Punycode': 'punycode'})

if (df_iana['tld'].apply(lambda tld: tld.count('.') > 1)).sum() > 0:
    raise 'Unexpected: TLDs should have only one point each.'

# cleaning TLDs from points and special right-to-left character
df_iana['tld'] = df_iana.tld.str.replace('.', '', n=1, regex=False)
df_iana['tld'] = df_iana.tld.str.replace('\u200f', '', n=1, regex=False)
df_iana['tld'] = df_iana.tld.str.replace('\u200e', '', n=1, regex=False)

# converting Type labels to IANA naming convention
df_tldlist['Type'] = df_tldlist['Type'].str.replace('gTLD', 'generic', regex=False)
df_tldlist['Type'] = df_tldlist['Type'].str.replace('ccTLD', 'country-code', regex=False)
df_tldlist['Type'] = df_tldlist['Type'].str.replace('grTLD', 'generic-restricted',regex=False)
df_tldlist['Type'] = df_tldlist['Type'].str.replace('sTLD', 'sponsored', regex=False)


df = df_iana.merge(df_tldlist, left_on='tld', right_on='TLD', how='outer')


### HANDLING NOT COINCIDENT TLDs ###

# check TLD-types should be the same execept for 'music' and pakistan پاكستان
if df[(~(df.Type == df['type']))].shape[0] > 2:
    print(df[(~(df.Type == df['type']))])
    raise 'Error: TLD-Types not coincided'

# check which IANA tlds are not in tldlist
iana_notin_tldlist = df[df.TLD.isna()].tld
if iana_notin_tldlist.shape[0] > 0:
    if not (iana_notin_tldlist.shape[0] == 1 and iana_notin_tldlist.iloc[0] == 'music'):
        print(f'Warning: IANA has {iana_notin_tldlist.shape[0]} TLDs not contained in tldlist:')
        for _, tmp in iana_notin_tldlist.iteritems():
            print(f'\t- {tmp}')
# check which tldlist tlds are not in IANA
tldlist_notin_iana = df[df.tld.isna()].TLD
if tldlist_notin_iana.shape[0] > 0:
    if not (tldlist_notin_iana.shape[0] == 1 and tldlist_notin_iana.iloc[0] == 'پاكستان'):
        print(f'Warning: tldlist has {tldlist_notin_iana.shape[0]} TLDs not contained in IANA')
        for _, tmp in tldlist_notin_iana.iteritems():
            print(f'\t- {tmp}')

# cloning not shared TLDs to specific columns (only one from pakistan)
nans = df.tld.isna()
df['tld'].values[nans] = df[nans].TLD
df['type'].values[nans] = df[nans].Type
df['manager'].values[nans] = df[nans].Sponsor

### FINISHED TO HANDLE NOT COINCIDENT TLDs ###

# creating a column with TLD and punycode when TLD is in a not-english like language
# punicode_isna = df['punycode'].isna()
# df['punycode'].values[punicode_isna] = df[punicode_isna].tld

df_tld = df[['tld', 'punycode', 'type', 'manager']].copy()

df_tld.head()

,tld,punycode,type,manager
0,aaa,NaN,generic,"American Automobile Association, Inc."
1,aarp,NaN,generic,AARP
2,abarth,NaN,generic,Fiat Chrysler Automobiles N.V.
3,abb,NaN,generic,ABB Ltd
4,abbott,NaN,generic,"Abbott Laboratories, Inc."


In [5]:
# getting the PSL list, considering the sections defined above, and parsing the comments.

with open('public_suffix_list.dat', 'r') as f:
    psl_lines = [ l.replace('\n', '') for l in f.readlines()]


sections_delimiters = [
    '// ===BEGIN ICANN DOMAINS===',
    '// newGTLDs',
    '// ===BEGIN PRIVATE DOMAINS==='
]
sections_names = [
    'icann',
    'icann-new',
    'private-domains'
]

regex_punycode = r'^\/\/ (xn--.*?) .*$'
regex_comment = r'^\/\/ (?!Submitted)(.*?)(?: : )(.*?)$'

line_start = 1 + psl_lines.index('// ===BEGIN ICANN DOMAINS===')

# Take attention to punycodes parsing: a new punycode should be used only for the PSL the punycode comment

sd = 0
manager = None
punycode = None
values = []
last_tld = ''
punycode_found = False
for i in range(line_start, len(psl_lines)):
    line = psl_lines[i]
    if len(line) == 0: continue
    if sd+1 < len(sections_delimiters) and line.find(sections_delimiters[sd+1]) == 0:
        sd += 1
    if line.find('//') == 0:
        punycode_match = re.match(regex_punycode, line)
        if punycode_match is not None:
            punycode_found = True
            punycode = punycode_match[1]
        else:
            first_comment_match = re.match(regex_comment, line)
            if first_comment_match is not None:
                manager = first_comment_match[1]
        continue
        
    tld = line
    tld = tld[tld.rfind('.')+1:]
    
    # if the tld (not the suffix) has changed and so the punycode too
    if last_tld != tld and not punycode_found:
        punycode = None
    punycode_found = False
    
    values.append([ sections_names[sd], tld, punycode, line, manager ])
    
    last_tld = tld
    pass

df_psl = pd.DataFrame(values, columns=['type', 'tld', 'punycode', 'suffix', 'manager'])

df_psl = df_psl[['type', 'tld', 'punycode', 'suffix', 'manager']].reset_index()

df_psl.head()

,index,type,tld,punycode,suffix,manager
0,0,icann,ac,None,ac,ac
1,1,icann,ac,None,com.ac,ac
2,2,icann,ac,None,edu.ac,ac
3,3,icann,ac,None,gov.ac,ac
4,4,icann,ac,None,net.ac,ac


In [8]:
# the merge will be done with the tld column

df = df_psl.reset_index().merge(df_tld.reset_index(), left_on='tld', right_on='tld', suffixes=['_psl', '_tld'], how='outer')

df['origin'] = 'both'
df['origin'].values[(~df['type_psl'].isna()) & (df['type_tld'].isna())] = 'PSL'
df['origin'].values[(~df['type_tld'].isna()) & (df['type_psl'].isna())] = 'ICANN'
df['origin'].values[(df['origin'] == 'both') & (df.tld != df.suffix)] = 'merged'

df = df.reset_index(drop=True)


# the 'type' columns:
# - equal to TLDLIST type
# - only if PSL type is private-domains, replace the previously defined TLDLIST type with the PSL one
# - for unknown types: other if punycode is empty, orphan-punycode otherwise
df['type'] = df['type_tld']

icann_pd = (df['type_psl'] == 'private-domains')
df['type'].values[icann_pd] = 'private-domains'


mask = (df['type_tld'].isna() & df['punycode_psl'].isna())
df['type'].values[mask] = 'other'

mask = (df['type_tld'].isna() & (~df['punycode_psl'].isna()))
df['type'].values[mask] = 'orphan-punycode'

# Checking rows having both punycodes not null matches
df_punycode = df[~(df.punycode_psl.isna()) & ~(df.punycode_tld.isna())]
if (df_punycode.punycode_psl != df_punycode.punycode_tld).sum() > 0:
    print('Warning: punycode does not match but should.')
    display(df_punycode[f_punycode.punycode_psl != df_punycode.punycode_tld])
df['punycode'] = df['punycode_psl']
notna_punycodetld = ~(df['punycode_tld'].isna())
df['punycode'].values[notna_punycodetld] = df['punycode_tld'].loc[notna_punycodetld]


df = df[[
    'suffix', 'tld',
    'punycode',
    'origin',
    'type', 'type_tld', 'type_psl',
    'manager_tld', 'manager_psl'
]]


df.rename(columns={'type_tld': 'tld type', 'type_psl': 'suffix type'}, inplace=True)
df.rename(columns={'manager_tld': 'tld manager', 'manager_psl': 'psl comment'}, inplace=True)

suffix_na = df.suffix.isna()
tld_na = df.tld.isna()

# checking empty tlds (should never happen)
if df[tld_na].shape[0] > 0:
    raise f'Error: {df[tld_na].shape[0]} NaN empty TLDs'
    
# checking empty suffixes
if df[suffix_na & tld_na].shape[0] > 0:
    print(f'Warning: there are {df[suffix_na].shape[0]} NaN Suffixes and TLDs')
df.suffix.values[suffix_na] = df.tld[suffix_na]

df = df.fillna('')

df.to_csv('tld_and_suffixes.csv')

df[df['origin'] != 'both'].to_csv('differents.csv')

codes = {
    'type': {
        'country-code': 'cc',
        'private-domains': 'pd',
        'sponsored': 'sp',
        'infrastructure': 'in',
        'generic-restricted': 'gr',
        'generic': 'ge',
        'other': 'ot',
        'orphan-punycode': 'op',
        'test': 'te'
    },
    'origin': {
        'both': 'b',
        'merged': 'm',
        'ICANN': 'i',
        'PSL': 'p'
    }
}

inv_codes = {}
for t in [ 'type', 'origin' ]:
    inv_codes[t] = {v: k for k, v in codes[t].items() }

df['code'] = df['origin'].apply(lambda o: codes['origin'][o]) + df.reset_index()['index'].astype(str) + df['type'].apply(lambda t: codes['type'][t])

df_etld = df.copy()

df_etld

,suffix,tld,punycode,origin,type,tld type,suffix type,tld manager,psl comment,code
0,ac,ac,,both,country-code,country-code,icann,Internet Computer Bureau Limited,ac,b0cc
1,com.ac,ac,,merged,country-code,country-code,icann,Internet Computer Bureau Limited,ac,m1cc
2,edu.ac,ac,,merged,country-code,country-code,icann,Internet Computer Bureau Limited,ac,m2cc
3,gov.ac,ac,,merged,country-code,country-code,icann,Internet Computer Bureau Limited,ac,m3cc
4,net.ac,ac,,merged,country-code,country-code,icann,Internet Computer Bureau Limited,ac,m4cc
...,...,...,...,...,...,...,...,...,...,...
9312,موبايلي,موبايلي,xn--mgbb9fbpob,ICANN,generic,generic,,Not assigned,,i9312ge
9313,珠宝,珠宝,xn--pbt977c,ICANN,generic,generic,,Not assigned,,i9313ge
9314,テスト,テスト,xn--zckzah,ICANN,test,test,,Internet Assigned Numbers Authority,,i9314te
9315,xperia,xperia,,ICANN,generic,generic,,Not assigned,,i9315ge


### Mask Boolean is not so fast
### Apply is slow compared with list comprehension
```

        tstart(f'get df {isp}')
        d = masks[tld][isp]
        tstop(f'get df {isp}')

        tstart(f'query {isp}')
        d.query('labels < @dn_max_suffix_labels')
        tstop(f'query {isp}')
        tstart(f'mask labels 1 {isp}')
        kk = d.labels <= dn_max_suffix_labels
        tstop(f'mask labels 1 {isp}')
        tstart(f'mask labels 2 {isp}')
        d = d[kk]
        tstop(f'mask labels 2 {isp}')
```

In [34]:
def getInvertedSuffixLabelSeries(df_etld):
    sfx = df_etld.suffix.str.replace(r'*.', '', regex=False).copy()
    maxLabels_suffix = sfx.str.count('\.').max()
    sfx = sfx.apply(lambda s: ('@@.'*(maxLabels_suffix - s.count('.'))) + s).str.split('.', expand=True)
    sfx = sfx[sfx.columns[::-1]]
    sfx = sfx.replace('@@', '')
    sfx = sfx.rename(columns={ 4:0, 3:1, 2:2, 1:3, 0:4})
    sfx['id'] = df.reset_index()['index']
    sfx['type'] = df['type']
    return sfx.copy()


In [87]:

class Node:
    def __init__(self, label, code=None, deep=0, fl=False, parent=None, dn=None):
        self.parent = parent
        self.label = label
        self.dn = dn
        self.code = code
        self.children = []
        self.deep = deep
        self.fl = fl
        self._debug = os.environ.get('DEBUG')
        pass
    
    def level(self):
        i = 0 if self.parent is None else self.parent.level() + 1 + self.fl
        return i
    
    def add(self, label, code=None, fl=False, dn=None):
        childDeep = self.deep+1 if not self.fl else self.deep
        node = Node(label, code=code, deep=childDeep, fl=fl, parent=self, dn=dn)
        self.children.append(node)
        return node
    
    def leaves(self):
        _leafs = [self] if self.code is not None else []
        for child in self.children:
            _leafs += child.leaves()
        return _leafs
    
    def allLeaf(self):
        return all([child.isLeaf() for child in self.children])
    
    def isLeaf(self):
        return self.code is not None
    
    def branch(self):
        if self.parent is None:
            return [ self ]
        return [ self ] + self.parent.branch()
    
    def __getitem__(self, key):
        return self.children[key]
    
    def compact(self):
        if self.parent is not None:
            while not self.fl and len(self.children) == 1 and not self.isLeaf():
                self.label = self.label + '\\.' + self[0].label
                self.dn = self[0].dn
                self.code = self[0].code
                self.fl = self[0].fl
                self.children = self[0].children
        for child in self.children:
            child.compact()
    
    def _print(self):
        print((self.level() * '  ') + str(self))
        for child in self.children:
            child._print()
    
    def __str__(self):
        if self.fl:
            return 'FL/' + self.label[0]
        else:
            pfl = int(self.parent.fl) if self.parent else ''
            return f'{self.label}#{pfl}' + f'[{self.dn}]'
    def __repr__(self):
        return self.__str__()
    
    def regexLongestSuffix(self, indent='  '):
        groups = []
        
        if self._debug:
            lindent = '\n' + self.level() * indent
            lindent2 = '\n' + (1 + self.level()) * indent
        else:
            lindent = ''
            lindent2 = ''
        
        for child in self.children:
            groups.append(child.regex(indent=indent))
        
        if self.parent is None:
            return '|'.join(groups)
        
        label = self.label[1:] if self.parent.fl else self.label
        
        leaf = f'(?P<{self.code}>{label})' if self.isLeaf() else ''
        
        if len(groups) == 0:
            return leaf
        
        bs = "" if self.fl else "\\."
        
        uncaptured = f'{label}{bs}'
        
        child_regex = f'|'.join(groups) 
        
        sep = ''
        if child_regex != '' and leaf != '':
            sep = '|'
        
        return f'{lindent}({uncaptured}({lindent2}{child_regex}{lindent2}){sep}{leaf})'

    def regex(self, indent='  '):
        groups = []
        
        if self._debug:
            lindent = '\n' + self.level() * indent
            lindent2 = '\n' + (1 + self.level()) * indent
        else:
            lindent = ''
            lindent2 = ''
        
        for child in self.children:
            groups.append(child.regex(indent=indent))
        
        if self.parent is None:
            return '|'.join(groups)
        
        label = self.label[1:] if self.parent.fl else self.label
        
        if self.isLeaf():
            label = f'(?P<{self.code}>{label})'
        
        if len(groups) == 0:
            return label
        
        bs = "" if self.fl else "\\."
        
        uncaptured = f'{label}{bs}'
        
        child_regex = f'|'.join(groups) 
        
        sep = ''
        if child_regex != '':
            sep = '|'
        
        bl = ''
        if self.deep == 0:
            bl = '^'
        
        return f'{bl}{lindent}(?:{uncaptured}(?:{lindent2}{child_regex}{sep}{lindent2}))'
    
    pass

def fillTree(sfxOr, l, parent):
    subcols = list(range(l+1, len(sfxOr.columns)-1)) # max number of labels is columns less 'index' column
    subcols.append('code')
    currentLabelUniques = sfxOr.drop_duplicates(subset=l)[l]
    firstLetters = currentLabelUniques.str[0].value_counts(sort=False)
    flNodes = {}
    for fl, c in firstLetters.iteritems():
        flNodes[fl] = parent.add(fl, fl=True) if c > 1 else parent
    for fl in flNodes:
        sfx = sfxOr[sfxOr[l].str[0] == fl]
        if l == 4:
            s_leaves = sfx
            s_branches = pd.DataFrame([])
        else:
            s_leaves = sfx[sfx[l+1] == '']
            s_branches = sfx[sfx[l+1] != ''][l].drop_duplicates()
        leaves = {}
        for _, leaf in s_leaves.iterrows():
            leaves[leaf[l]] = flNodes[leaf[l][0]].add(leaf[l], code=leaf['code'], dn='.'.join(leaf.values[:l+1]))
        for loc, branch in s_branches.iteritems():
            if branch not in leaves:
                node = flNodes[branch[0]].add(branch)
            else:
                node = leaves[branch]
            if (sfx[l+1] != '').sum() > 0:
                fillTree(sfx[(sfx[l] == branch) & (sfx[l+1] != '')], l+1, node)
        pass

def invertedSuffixLabels(df_etld):
    sfx = df_etld.suffix.str.replace(r'*.', '', regex=False).copy()
    maxLabels_suffix = sfx.str.count('\.').max()
    sfx = sfx.apply(lambda s: ('@@.'*(maxLabels_suffix - s.count('.'))) + s).str.split('.', expand=True)
    sfx = sfx[sfx.columns[::-1]]
    sfx = sfx.replace('@@', '')
    sfx = sfx.rename(columns={ 4:0, 3:1, 2:2, 1:3, 0:4})
    sfx['code'] = df_etld['code']
    return sfx.copy()

regexes = {}
def getRegexes(df_etld):
    sfx = invertedSuffixLabels(df_etld)
    tlds = sfx[0].drop_duplicates()
    for tld in tlds:
        tree = Node('root', deep=-1)
        fillTree(sfx[sfx[0] == tld], 0, tree)
        tree.compact()
        if os.environ.get('DEBUG'):
            regexes[tld] = re.compile(tree.regex(), re.VERBOSE)
        else:
            regexes[tld] = re.compile(tree.regex())
    return regexes

regexes = getRegexes(df_etld[df_etld.tld == 'cn'])

#print(regexes['com'].pattern)

In [88]:
def match(s):
    print(regexes[s[0]].pattern)
    gd = regexes[s[0]].match(s[1]).groupdict()
    print([ g for g in gd if gd[g] is not None])
    gd1 = { inv_codes['type'][k[-2:]]: int(k[1:-2]) for k in gd if gd[k] is not None }
    gd2 = { inv_codes['origin'][k[0]]: int(k[1:-2]) for k in gd if gd[k] is not None }
    gd3 = [ [ int(k[1:-2]), inv_codes['type'][k[-2:]], inv_codes['origin'][k[0]] ] for k in gd if gd[k] is not None ]
    return gd3

def single(dn):
    dn = dn.split('.')[::-1]

    start = time.time()
    m = match((dn[0], '.'.join(dn)))
    if m is None:
        raise 'Not found'
    end = time.time() - start

    print(f'Found {len(m)} solution/s in {end} sec')

    return m

single('ciao.cn-north-1.eb.amazonaws.com.cn')


^(?:(?P<b733cc>cn)\.(?:(?:a(?:(?P<m734cc>c)|(?P<m744cc>h)|))|(?:c(?:(?:(?P<m735cc>om)\.(?:(?:amazonaws\.(?:(?:c(?:(?P<m778pd>ompute)|(?P<m782pd>n-north-1\.s3)|))|(?:e(?:(?P<m781pd>lb)|(?:b\.c(?:(?P<m779pd>n-north-1)|(?P<m780pd>n-northwest-1)|))|))|))|))|(?P<m746cc>q)|))|(?:g(?:(?P<m737cc>ov)|(?P<m748cc>d)|(?P<m749cc>s)|(?P<m750cc>z)|(?P<m751cc>x)|))|(?:n(?:(?P<m738cc>et)|(?P<m762cc>m)|(?P<m763cc>x)|))|(?:m(?:(?P<m740cc>il)|(?P<m776cc>o)|))|(?:h(?:(?P<m752cc>a)|(?P<m753cc>b)|(?P<m754cc>e)|(?P<m755cc>i)|(?P<m756cc>l)|(?P<m757cc>n)|(?P<m775cc>k)|))|(?:j(?:(?P<m758cc>l)|(?P<m759cc>s)|(?P<m760cc>x)|))|(?:s(?:(?P<m765cc>c)|(?P<m766cc>d)|(?P<m767cc>h)|(?P<m768cc>n)|(?P<m769cc>x)|))|(?:t(?:(?P<m770cc>j)|(?P<m777cc>w)|))|(?:x(?:(?P<m771cc>j)|(?P<m772cc>z)|))|(?P<m736cc>edu)|(?P<m739cc>org)|(?P<m741cc>公司)|(?P<m742cc>网络)|(?P<m743cc>網絡)|(?P<m745cc>bj)|(?P<m747cc>fj)|(?P<m761cc>ln)|(?P<m764cc>qh)|(?P<m773cc>yn)|(?P<m774cc>zj)|(?P<m783pd>instantcloud)|))
['b733cc', 'm735cc', 'm779pd']
Found 3 soluti

[[733, 'country-code', 'both'],
 [735, 'country-code', 'merged'],
 [779, 'private-domains', 'merged']]

In [20]:
def example_mergeSuffixes(regexes, frame, df_etld):
    start = time.time()
    dnRevertedSeries = frame.dn.str.split('.').apply(lambda s: [ s[-1] , '.'.join(s[::-1]) ])

    def match(s):
        if s[0] not in regexes:
            return -1
        gd = regexes[s[0]].match(s[1]).groupdict()
        gd = [ int(k[1:]) for k in gd if gd[k] is not None ]
        return gd[0] if len(gd) == 1 else gd

    matchesSeries = dnRevertedSeries.apply(match)
    mergeFrame = matchesSeries.to_frame().merge(df_etld, left_on='dn', right_index=True, how='left').drop(columns='dn')
    result = frame.dn.to_frame().join(mergeFrame).copy()
    end = time.time() - start
    print(f'Merged {datasetFrame.shape[0]} in {end} sec ({end/datasetFrame.shape[0]} sec/dn)')
    return result

example_mergeSuffixes(regexes, datasetFrame, df_etld)

Merged 674898 in 25.887436389923096 sec (3.835755386728527e-05 sec/dn)


,dn,suffix,tld,punycode,origin,type,tld type,suffix type,tld manager,psl comment
0,mortiscontrastatim.com,com,com,,both,generic,generic,icann,VeriSign Global Registry Services,com
1,cvyh1po636avyrsxebwbkn7.ddns.net,ddns.net,net,,merged,orphan-punycode,generic,private-domains,VeriSign Global Registry Services,No-IP.com
2,plasticbags.sa.com,sa.com,com,,merged,orphan-punycode,generic,private-domains,VeriSign Global Registry Services,CentralNic
3,mzltrack.com,com,com,,both,generic,generic,icann,VeriSign Global Registry Services,com
4,miss-slim.ru,ru,ru,,both,country-code,country-code,icann,Coordination Center for TLD RU,ru
...,...,...,...,...,...,...,...,...,...,...
674893,fkmmnlmdokedddmn.com,com,com,,both,generic,generic,icann,VeriSign Global Registry Services,com
674894,cwnhlchfhcqrgass.eu,eu,eu,,both,country-code,country-code,icann,EURid vzw/asbl,eu
674895,weloty.com,com,com,,both,generic,generic,icann,VeriSign Global Registry Services,com
674896,mwxquxbokithpn.net,net,net,,both,generic,generic,icann,VeriSign Global Registry Services,net


In [100]:
df_etld['level'] = df_etld.suffix.str.count('\.')

df_etld.drop_duplicates(subset=['level', 'origin', 'type']).sort_values(by='level')[['suffix', 'level', 'origin', 'type']]

,suffix,level,origin,type
0,ac,0,both,country-code
9212,active,0,ICANN,generic
7717,ايران,0,PSL,orphan-punycode
6206,onion,0,PSL,other
809,com,0,both,generic
343,biz,0,both,generic-restricted
9215,an,0,ICANN,country-code
9298,测试,0,ICANN,test
166,arpa,0,both,infrastructure
19,aero,0,both,sponsored
